In [5]:
pip install transformers torch datasets

In [6]:
from transformers import BlipProcessor, BlipForQuestionAnswering
import torch


processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")


In [7]:
from PIL import Image

image = Image.open("/content/Cat Medley_ Cuteness Galore, Funnies, Rescues, And Appreciation.jpg").convert("RGB")
question = "What is the color of the cat?"


inputs = processor(
    images=image,
    text=question,
    return_tensors="pt"
)

In [8]:
with torch.no_grad():
    outputs = model.generate(**inputs, max_length=20)


answer = processor.decode(outputs[0], skip_special_tokens=True)
print(f"Question: {question}")
print(f"Answer: {answer}")

Question: What is the color of the cat?
Answer: white


In [9]:
outputs = model.generate(
    **inputs,
    max_length=20,
    num_beams=5,
    early_stopping=True,
    temperature=0.7,
)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [10]:
pip install datasets

In [11]:
from datasets import load_dataset

# Load OK-VQA dataset
dataset = load_dataset("HuggingFaceM4/OK-VQA", split="train")  # Use "validation" for eval

README.md:   0%|          | 0.00/127 [00:00<?, ?B/s]

OK-VQA.py:   0%|          | 0.00/5.98k [00:00<?, ?B/s]

The repository for HuggingFaceM4/OK-VQA contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/HuggingFaceM4/OK-VQA.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Repo card metadata block was not found. Setting CardData to empty.


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [12]:
def preprocess(examples):
    images = [img.convert("RGB") for img in examples["image"]]
    questions = examples["question"]
    answers = [example_answers[0]['answer'] for example_answers in examples["answers"]]


    inputs = processor(
        text=questions,
        images=images,
        padding=True,
        truncation=True,
        return_tensors="pt",
    )


    labels = processor.tokenizer(
        answers,
        padding=False,
        truncation=True,
        return_tensors="pt",
    ).input_ids

    inputs["labels"] = labels
    return inputs


In [22]:
processed_dataset = dataset.map(
    preprocess,
    batched=True,
    batch_size=32,
    remove_columns=dataset.column_names,
)

Map:   0%|          | 0/9009 [00:00<?, ? examples/s]

OSError: [Errno 28] No space left on device

In [23]:
import os
from google.colab import files

# List files
!ls

 blip-vqa-finetuned
'Cat Medley_ Cuteness Galore, Funnies, Rescues, And Appreciation.jpg'
 sample_data


In [15]:
processed_dataset.set_format("torch", columns=["pixel_values", "input_ids", "attention_mask", "labels"])

In [16]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding,  DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(
    processor.tokenizer,
    model=model,
    padding="longest",
)

In [17]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./blip-vqa-finetuned",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",
    save_strategy="epoch",
    report_to="none"
)

In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset,

)


trainer.train()

RuntimeError: stack expects each tensor to be equal size, but got [18] at entry 0 and [20] at entry 1